In [95]:
# imports
from PIL import Image
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


In [96]:
# load dataset
numbers = [2, 3, 4]
pics = 100
image_arrays = []
for i in numbers:
    for j in range(pics):
        file_path = os.path.join('dataset', str(i), f'{i}_{j}.png')
        if os.path.exists(file_path):
            image = Image.open(file_path)
            image_array = np.array(image)
            image_array_red_channel= image_array[:, :, 0]
            image_array_red_channel = image_array_red_channel.flatten()
            image_arrays.append(image_array_red_channel)

number = []
for i in numbers:
    for j in range(pics):
        number.append(i)

df = pd.DataFrame(image_arrays)
df=df/255
df['number'] = number


In [97]:
# train and test split
df_random = df.sample(frac=1, random_state=42)
split = int(0.8*len(df_random))
df_train = df_random.iloc[:split]
df_test = df_random.iloc[split:]

x_train = df_train.drop('number', axis=1).values
y_train = df_train['number'].values
y_train_encoded = to_categorical(y_train-2)
x_test = df_test.drop('number', axis=1).values
y_test = df_test['number'].values
y_test_encoded = to_categorical(y_test-2)


In [183]:
# model
model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation='relu'))  # Input layer
model.add(Dense(16, activation='relu'))  # Hidden layer
model.add(Dense(3, activation='softmax'))  # Output layer

In [184]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

In [185]:
early_stopping = EarlyStopping(monitor='val_loss',patience=20)
# Train the model
model.fit(x_train, y_train_encoded, validation_data=(x_test, y_test_encoded), epochs=150, batch_size=8, callbacks=[early_stopping])

Epoch 1/150
30/30 [==============================] - 1s 7ms/step - loss: 1.1600 - accuracy: 0.3542 - precision: 0.2540 - recall: 0.0667 - val_loss: 1.0642 - val_accuracy: 0.5500 - val_precision: 0.5000 - val_recall: 0.0167
Epoch 2/150
30/30 [==============================] - 0s 2ms/step - loss: 0.9725 - accuracy: 0.5583 - precision: 0.8235 - recall: 0.1750 - val_loss: 1.0009 - val_accuracy: 0.4167 - val_precision: 0.3778 - val_recall: 0.2833
Epoch 3/150
30/30 [==============================] - 0s 2ms/step - loss: 0.8757 - accuracy: 0.6333 - precision: 0.7109 - recall: 0.3792 - val_loss: 0.8349 - val_accuracy: 0.7333 - val_precision: 0.8750 - val_recall: 0.4667
Epoch 4/150
30/30 [==============================] - 0s 2ms/step - loss: 0.7302 - accuracy: 0.7292 - precision: 0.8068 - recall: 0.5917 - val_loss: 0.9945 - val_accuracy: 0.5667 - val_precision: 0.5741 - val_recall: 0.5167
Epoch 5/150
30/30 [==============================] - 0s 2ms/step - loss: 0.7495 - accuracy: 0.6500 - precisi

In [186]:
# Evaluate the model
loss, accuracy, precision, recall = model.evaluate(x_test, y_test_encoded)
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('Loss: %.2f' % loss)

2/2 [==============================] - 0s 5ms/step - loss: 0.4318 - accuracy: 0.8667 - precision: 0.8667 - recall: 0.8667
Accuracy: 86.67
Precision: 86.67
Recall: 86.67
Loss: 0.43
